# Near Decision Boundry Sample Removal

### Reading the Data built by the Previous Notebooks

In [1]:
import pickle
import numpy as np

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/3_KnnImputation/"
imputed_data = []

number_of_partitions = 5
for i in range(1,number_of_partitions +1):
# Load the array with Pickle
    with open(data_dir + "Partition" + str(i) + "_KnnImputation" + ".pkl", 'rb') as f:
        imputed_data.append(pickle.load(f))

In [2]:
import pandas as pd
labels = []
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/2_Labels/"

for i in range(1,6):
    labels.append(pd.read_csv(data_dir + "Partition" + str(i) + "_labels.csv"))

## Function to Remove Samples

In [5]:
# NewFeatures with Shuffle
from tqdm import tqdm

def dropping(start_partition, end_partition, data_dir, data, labels, classes_to_filter, category_mapping, name):


    num_attributes = 25
    num_timestamps = 60       

    for i in range(start_partition-1,end_partition):
        
        filtered_y_train_df = labels[i][~labels[i]['FLARE_CLASS'].isin(classes_to_filter)]
        filtered_indices = filtered_y_train_df.index
        x_train_filtered = data[i][:, :, filtered_indices]
        
        num_samples = np.array(x_train_filtered).shape[2]
        X_train = np.zeros((num_samples, num_timestamps*(num_attributes-1)))
        Y_train = np.zeros(num_samples)
        
        the_labels = pd.DataFrame()
        the_labels['FLARE_CLASS'] = filtered_y_train_df['FLARE_CLASS'].map(category_mapping)
        each_partition = np.zeros((num_timestamps, num_attributes, num_samples))
        each_partition = np.array(x_train_filtered)

        with tqdm(num_samples) as pbar:
            for j in range(0, num_samples):
                new_column = np.zeros((num_timestamps,num_attributes)) 
                new_column = each_partition[:,:,j]
                
                flettened = np.zeros(num_timestamps*(num_attributes-1))

                for m in range(1,num_attributes):
                    flettened[(m-1)*num_timestamps:m*num_timestamps] = new_column[:,m]

                X_train[j, :] = flettened
                Y_train[j] = the_labels['FLARE_CLASS'].iloc[j]
                
                pbar.update(1)

        print("P"+str(i+1)+ " "+str(X_train.shape) +" Nan-Value: "+ str(np.isnan(X_train).any()))     
        
        num_samples = X_train.shape[0]
        shuffle_indices = np.random.permutation(num_samples)

        X_train_shuffled = X_train[shuffle_indices]
        Y_train_shuffled = Y_train[shuffle_indices]
    

        with open(data_dir + "Partition" + str(i+1)
                       + name + "_Concatenation_KnnImputation" + ".pkl", 'wb') as f:
            pickle.dump(X_train_shuffled, f)

        with open(data_dir + "Partition" + str(i+1)
                       + "_Labels" + name + "_Concatenation_KnnImputation" + ".pkl", 'wb') as f:
            pickle.dump(Y_train_shuffled, f)

### Removing B and C classes 

In [6]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_3_FinalData_WithoutB&C_Concatenation_KnnImputation/"

classes_to_filter = ['C', 'B']
category_mapping = {'X': 1, 'M': 1,'FQ': 0}

name = "_WithoutB&C"
    
dropping(1,5,data_dir,imputed_data,labels, classes_to_filter, category_mapping, name)

61384it [00:00, 78478.59it/s]


P1 (61384, 1440) Nan-Value: False


74769it [00:00, 77303.01it/s]


P2 (74769, 1440) Nan-Value: False


36186it [00:00, 75535.28it/s]


P3 (36186, 1440) Nan-Value: False


44459it [00:00, 78802.18it/s]


P4 (44459, 1440) Nan-Value: False


63678it [00:00, 78424.47it/s]


P5 (63678, 1440) Nan-Value: False


### Removing C class

In [7]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_4_FinalData_WithoutC_Concatenation_KnnImputation/"

classes_to_filter = ['C']
category_mapping = {'X': 1, 'M': 1, 'B': 0, 'FQ': 0}

name = "_WithoutC"

dropping(1,5,data_dir,imputed_data,labels, classes_to_filter, category_mapping, name)

67076it [00:00, 79367.88it/s]


P1 (67076, 1440) Nan-Value: False


79747it [00:01, 78047.28it/s]


P2 (79747, 1440) Nan-Value: False


36871it [00:00, 74525.87it/s]


P3 (36871, 1440) Nan-Value: False


45305it [00:00, 74981.33it/s]


P4 (45305, 1440) Nan-Value: False


69602it [00:00, 79179.67it/s]


P5 (69602, 1440) Nan-Value: False


## Vector Data

In [15]:
# NewFeatures with Shuffle

def new_Features_pkl(start_partition, end_partition, data_dir, data, labels, name):

    number_of_new_features = 9
    num_attributes = 24
    num_timestamps = 60
    
    for i in range(start_partition-1,end_partition):
        
        each_partition = np.zeros((data[i].shape[0], num_timestamps*(num_attributes)))
        each_partition = np.array(data[i])
        new_partition = np.zeros((each_partition.shape[0], number_of_new_features*num_attributes))
        
        with tqdm(each_partition.shape[0]) as pbar:
            for j in range(0,each_partition.shape[0]):
                new_column = np.zeros((num_timestamps*num_attributes)) 
                new_column = each_partition[j,:]
                

                new_features = np.zeros(number_of_new_features*(num_attributes))

                for m in range(0,num_attributes):
                    
                    mean = np.mean(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    new_features[((m)*number_of_new_features) + 0] = mean
                    median = np.median(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    new_features[((m)*number_of_new_features) + 1] = median
                    std = np.std(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    new_features[((m)*number_of_new_features) + 2] = std
                    
                    skewness = skew(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    if skewness == np.nan:
                        skewness = new_partition[j-1, ((m)*number_of_new_features) + 3]
                    new_features[((m)*number_of_new_features) + 3] = skewness
                    
                    kurtosis_value = kurtosis(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    if kurtosis_value == np.nan:
                        kurtosis_value = new_partition[j-1, ((m)*number_of_new_features) + 4]
                    new_features[((m)*number_of_new_features) + 4] = kurtosis_value
                    
                    indices = np.arange(num_timestamps)
                    weight_array = indices / num_timestamps
                    weighted_avg = np.average(new_column[m*num_timestamps:(m+1)*num_timestamps], weights=weight_array)
                    if weighted_avg == np.nan:
                        weighted_avg = new_partition[j-1, ((m)*number_of_new_features) + 5]
                    new_features[((m)*number_of_new_features) + 5] = weighted_avg
                    
                    last_value = new_column[(m+1)*num_timestamps-1]
                    new_features[((m)*number_of_new_features) + 6] = last_value
                    first_value = new_column[m*num_timestamps]
                    new_features[((m)*number_of_new_features) + 7] = first_value
                    
                    numerator = np.sum((new_column[m*num_timestamps:(m+1)*num_timestamps] - mean) * (indices - np.mean(indices)))
                    denominator = np.sum((new_column[m*num_timestamps:(m+1)*num_timestamps] - mean) ** 2)
                    slope = numerator / denominator
                    if slope == np.nan:
                        slope = new_partition[j-1, ((m)*number_of_new_features) + 8]
                    new_features[((m)*number_of_new_features) + 8] = slope
                
                    
                new_partition[j,:] = new_features
                
                pbar.update(1)
                

        print("P"+str(i+1)+" Nan-Value: "+ str(np.isnan(new_partition).any()))  
        X_train = new_partition
        Y_train = labels[i]


        num_samples = X_train.shape[0]
        shuffle_indices = np.random.permutation(num_samples)

        X_train_shuffled = X_train[shuffle_indices]
        Y_train_shuffled = Y_train[shuffle_indices]
    

        with open(data_dir + "Partition" + str(i+1) 
                       + name + ".pkl", 'wb') as f:
            pickle.dump(X_train_shuffled, f)

        with open(data_dir + "Partition" + str(i+1) 
                       + "_Labels" + name + ".pkl", 'wb') as f:
            pickle.dump(Y_train_shuffled, f)

### Removing B and C classes 

In [14]:
import numpy as np
import pandas as pd
import pickle

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_3_FinalData_WithoutB&C_Concatenation_KnnImputation/"

imputed_wBandC_data = []
wBandC_labels = []


num_partitions = 5

for i in range(0,num_partitions):
# Load the array with Pickle
    with open(data_dir + "Partition" + str(i+1) + "_WithoutB&C_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        imputed_wBandC_data.append(pickle.load(f))
        
    print("P"+str(i+1)+" Nan-Value: "+ str(np.isnan(imputed_wBandC_data[i]).any() or np.isinf(imputed_wBandC_data[i]).any()))  

    with open(data_dir + "Partition" + str(i+1) + "_Labels_WithoutB&C_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        wBandC_labels.append(pickle.load(f))

P1 Nan-Value: False
P2 Nan-Value: False
P3 Nan-Value: False
P4 Nan-Value: False
P5 Nan-Value: False


In [19]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_6_FinalData_WithoutB&C_NewFeatures_KnnImputation/"

name = '_WithoutB&C_KnnImputation'
new_Features_pkl(1, 5, data_dir, imputed_wBandC_data, wBandC_labels, name)

61384it [06:26, 159.02it/s]


P1 Nan-Value: False


74769it [07:49, 159.11it/s]


P2 Nan-Value: False


36186it [03:46, 159.56it/s]


P3 Nan-Value: False


44459it [04:39, 159.34it/s]


P4 Nan-Value: False


63678it [06:37, 160.07it/s]


P5 Nan-Value: False


### Removing C class

In [20]:
import numpy as np
import pandas as pd
import pickle

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_4_FinalData_WithoutC_Concatenation_KnnImputation/"

imputed_wC_data = []
wC_labels = []


num_partitions = 5

for i in range(0,num_partitions):
# Load the array with Pickle
    with open(data_dir + "Partition" + str(i+1) + "_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        imputed_wC_data.append(pickle.load(f))
        
    print("P"+str(i+1)+" Nan-Value: "+ str(np.isnan(imputed_wC_data[i]).any() or np.isinf(imputed_wC_data[i]).any()))  

    with open(data_dir + "Partition" + str(i+1) + "_Labels_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        wC_labels.append(pickle.load(f))

P1 Nan-Value: False
P2 Nan-Value: False
P3 Nan-Value: False
P4 Nan-Value: False
P5 Nan-Value: False


In [21]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_7_FinalData_WithoutC_NewFeatures_KnnImputation/"

name = '_WithoutC_KnnImputation'
new_Features_pkl(1, 5, data_dir, imputed_wC_data, wC_labels, name)

67076it [06:56, 161.03it/s]


P1 Nan-Value: False


79747it [08:18, 160.04it/s]


P2 Nan-Value: False


36871it [03:50, 159.80it/s]


P3 Nan-Value: False


45305it [04:41, 160.80it/s]


P4 Nan-Value: False


69602it [07:13, 160.55it/s]


P5 Nan-Value: False
